In [1]:
import os
import pickle
import numpy as np
from scipy.io import loadmat

import scfw.cov_estimation as ce
from scfw.frank_wolfe import frank_wolfe
from scfw.scopt import scopt

In [2]:
data_folder = '../data/'
problem_name = 'Lymph'
results = {problem_name: {}}
results_folder = './results/'
results_file = os.path.join(results_folder, 'cov_estimation', problem_name + '.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results = pickle.load(f)
        except Exception:
            pass 

In [3]:
def sample_l1_point(r, ndim):
    vec = np.random.rand(ndim,)
    vec /= np.linalg.norm(vec, ord=1)
    return vec

In [4]:
# load data
data = loadmat(os.path.join(data_folder, problem_name + '.mat'))
    
A = data['S']
p = A.shape[0]
r = 1300
X_0 = r * np.eye(p) / p
#X_0 = np.diag(sample_l1_point(r, p))

## Parameters initialization

In [5]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-10

FW_params={
    'iter_FW': 10000,
    'line_search_tol': 1e-5,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Functions

In [6]:
func_x = lambda X: ce.cov_val(A, X)
func_beta = lambda X, S, beta, extra_param, extra_param_s: ce.cov_val(A, (1 - beta) * X + beta * S)
grad_x = lambda X, inv_X: ce.cov_grad(A, X, inv_X)
grad_beta = lambda X, S, beta, inv_X, inv_S: ce.cov_grad(A, (1 - beta) * X + beta * S) # TODO: add inverse
hess_mult_x = lambda S, inv_X: ce.cov_hess_mult(S, inv_X)
hess_mult_vec_x = lambda S, inv_X: ce.cov_hess_mult_vec(S, inv_X)
#extra_func = lambda X: np.linalg.eigvalsh(X)
extra_func = lambda X: X
linear_oracle = lambda grad: ce.linear_oracle(grad, r)
prox_func = lambda X, L: ce.proj_map(X) #used for SCOPT

## Run FW

In [7]:
#run_alpha_policies = ['standard', 'line_search', 'backtracking', 'sc', 'sc_backtracking', 'sc_hybrid']
run_alpha_policies = ['sc', 'sc_backtracking', 'sc_hybrid']

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=100, 
                       debug_info=False)
    
    results[problem_name + '.mat'][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
    with open('./results/cov_estimation/lymph.pckl', 'wb') as f:
        pickle.dump(results, f)

und=212.64366555455652, real_Gap=8.78e+01, f_val=300.3956020180625
iter=7500, stepsize=3.12e-05, criterion=1.37e-03, upper_bound=300.18577544204595, lower_bound=213.6493524738425, real_Gap=8.65e+01, f_val=300.18577544204595
iter=7600, stepsize=6.13e-05, criterion=1.34e-03, upper_bound=299.9921537690954, lower_bound=215.0981880570091, real_Gap=8.49e+01, f_val=299.9921537690954
iter=7700, stepsize=6.11e-05, criterion=1.31e-03, upper_bound=299.7959866061034, lower_bound=217.49283372712432, real_Gap=8.23e+01, f_val=299.7959866061034
iter=7800, stepsize=3.07e-05, criterion=1.29e-03, upper_bound=299.6015423600858, lower_bound=218.43745264400542, real_Gap=8.12e+01, f_val=299.6015423600858
iter=7900, stepsize=3.12e-05, criterion=1.29e-03, upper_bound=299.42955505841564, lower_bound=218.43745264400542, real_Gap=8.10e+01, f_val=299.42955505841564
iter=8000, stepsize=5.97e-05, criterion=1.28e-03, upper_bound=299.24971437100595, lower_bound=218.83995264248915, real_Gap=8.04e+01, f_val=299.24971437

In [10]:
with open('./results/cov_estimation/lymph.pckl', 'wb') as f:
    pickle.dump(results, f)

In [29]:
results['Lymph.mat'].keys()

dict_keys(['standard', 'backtracking'])